# ComfyUI QuickStart 📓

> Minimal, config-driven installer for ComfyUI models and custom nodes.

---
30-python3.12-cuda12.1.1-torch2.5.1 template and use it in the Jupyter Notebook (port: 8888). <br>
Link to template:  [ComfyUI v.0.3.30](https://runpod.io/console/deploy?template=fnox0jr8pw&ref=9n2q5pa8)
Remember to Increase Pod Volumne to above 60gbs and 170gbs for ephemeral 
)
This notebook is designed to be:
- Config-driven (add HF/CivitAI/custom nodes in one CONFIG object)
- Robust (retries, clear logs, token-aware)
- Portable (no hardcoded model lists)

Edit CONFIG, run Apply, verify. That's it.


In [ ]:
# Tokens and Config
# Fill in tokens or leave empty to download public assets anonymously
# TODO: get as env variables
HF_TOKEN = ""
CIVITAI_TOKEN = ""

# UI
SHOW_PROGRESS = True  # stream command output and show curl progress bars (WIP)
DRY_RUN = False       # set True to print plan without downloading
VALIDATE_SIZES = False  # attempt to fetch sizes before downloading (slower)
SKIP_EXISTING = True    # skip files that already exist (size > 0)
INSTALL_NODE_REQUIREMENTS = True  # install requirements.txt in cloned custom nodes if present

# Config-driven downloads
CONFIG = {
    # HuggingFace: list of items with repo/file and destination dir
    # dest can be a key in CONFIG["dirs"] (recommended) or an absolute path
    "huggingface_models": [
        # FLUX.1-dev core
        {"repo": "black-forest-labs/FLUX.1-dev", "files": ["ae.safetensors"], "dest": "vae"},
        {"repo": "black-forest-labs/FLUX.1-dev", "files": ["flux1-dev.safetensors"], "dest": "unet"},
        {"repo": "comfyanonymous/flux_text_encoders", "files": ["t5xxl_fp16.safetensors", "clip_l.safetensors"], "dest": "clip"},

        # GGUF UNET (Fill)
        {"repo": "YarvixPA/FLUX.1-Fill-dev-GGUF", "files": ["flux1-fill-dev-Q6_K.gguf"], "revision": "ed1b97dfd7faedbeaac08635330dc3ed23e86ea1", "dest": "unet"},

        # Dual CLIP
        {"repo": "comfyanonymous/flux_text_encoders", "files": ["clip_l.safetensors", "t5xxl_fp16.safetensors"], "dest": "clip"},

        # CLIP Vision
        {"repo": "Comfy-Org/sigclip_vision_384", "files": ["sigclip_vision_patch14_384.safetensors"], "dest": "clip_vision"},

        # Style Model
        {"repo": "black-forest-labs/FLUX.1-Redux-dev", "files": ["flux1-redux-dev.safetensors"], "dest": "style_models"},

        # VAE (pinned revision)
        {"repo": "lovis93/testllm", "files": ["ae.safetensors"], "revision": "ed9cf1af7465cebca4649157f118e331cf2a084f", "dest": "vae"},

        # LoRA from HF
        {"repo": "ali-vilab/ACE_Plus", "files": ["portrait/comfyui_portrait_lora64.safetensors"], "dest": "loras"},

        # SAM2
        {"repo": "facebook/sam2-hiera-large", "files": ["sam2_hiera_large.pt"], "dest": "sam2"},

        # Optional CLIP variant (may be unavailable)
        # {"repo": "zer0int/CLIP-GmP-ViT-L-14", "files": ["ViT-L-14-TEXT-detail-improved-hiT-GmP-TE-only-HF.safetensors"], "dest": "clip"},

        # Optional: FLUX.1-Turbo-Alpha UNET
        # {"repo": "alimama-creative/FLUX.1-Turbo-Alpha", "files": ["diffusion_pytorch_model.safetensors"], "dest": "unet"},
    ],

    # CivitAI: supply version IDs or model IDs; type determines destination
    # You can also override destination (dest) and filename per-item
    "civitai_items": [
        # {"id": "0000000", "type": "checkpoint", "filename": "example_mid.safetensors"},
        # {"id": "0000000", "type": "lora"},
    ],

    # Custom nodes: git repos to clone or update
    "custom_nodes": [
        {"repo": "https://github.com/ltdrdata/ComfyUI-Manager.git"},
        # Add more as needed, e.g. Impact Pack:
        # {"repo": "https://github.com/ltdrdata/ComfyUI-Impact-Pack.git"},
    ],

    # Base directories for ComfyUI
    "dirs": {
        "checkpoints": "/workspace/ComfyUI/models/checkpoints",
        "unet": "/workspace/ComfyUI/models/unet",
        "clip": "/workspace/ComfyUI/models/clip",
        "clip_vision": "/workspace/ComfyUI/models/clip_vision",
        "style_models": "/workspace/ComfyUI/models/style_models",
        "vae": "/workspace/ComfyUI/models/vae",
        "loras": "/workspace/ComfyUI/models/loras",
        "embeddings": "/workspace/ComfyUI/models/embeddings",
        "sam2": "/workspace/ComfyUI/models/sam2",
        "custom_nodes": "/workspace/ComfyUI/custom_nodes",
    }
}

print("✅ CONFIG ready. Edit CONFIG above to suit your needs.")


In [ ]:
# Helpers
import os, subprocess, shutil, json, time, re
import requests

def run(cmd: str, timeout: int = 1800) -> tuple[bool, str]:
    # For curl, show progress by not capturing output. For others, capture.
    wants_stream = SHOW_PROGRESS and (cmd.strip().startswith("curl ") or " curl " in cmd)
    if wants_stream:
        print(f"$ {cmd}")
        try:
            p = subprocess.run(cmd, shell=True, text=True, timeout=timeout)
            ok = (p.returncode == 0)
            return ok, ""
        except subprocess.TimeoutExpired:
            return False, f"timeout after {timeout}s: {cmd}"
    else:
        try:
            p = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=timeout)
            out = (p.stdout or '').strip()
            err = (p.stderr or '').strip()
            if p.returncode == 0:
                if out:
                    print(out)
                return True, out
            else:
                if err:
                    print(err)
                return False, err
        except subprocess.TimeoutExpired:
            return False, f"timeout after {timeout}s: {cmd}"


def ensure_dirs():
    d = CONFIG.get("dirs", {})
    all_dirs = list(d.values())
    if all_dirs:
        run("mkdir -p " + " ".join(all_dirs))


def ensure_hf_cli():
    # CLI and faster transfer backend (shows progress)
    run("pip install -q --upgrade huggingface_hub[cli] hf_transfer")
    os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'
    # Create alias if only huggingface-cli exists
    hf = shutil.which('hf') or shutil.which('huggingface-cli')
    if not shutil.which('hf') and hf:
        run(f"ln -sf {hf} /usr/local/bin/hf || true")
    # Auth if token is set
    if HF_TOKEN:
        os.environ['HUGGINGFACE_HUB_TOKEN'] = HF_TOKEN
        os.environ['HF_TOKEN'] = HF_TOKEN
        run("hf whoami || true")


def file_exists_nonempty(path: str) -> bool:
    try:
        return os.path.isfile(path) and os.path.getsize(path) > 0
    except Exception:
        return False


def hf_download(repo: str, files: list[str], dest: str, revision: str | None = None) -> None:
    run(f"mkdir -p '{dest}'")
    # Prefer curl for progress; fallback to hf CLI on errors
    for fpath in files:
        used_curl = False
        rev = revision or "main"
        url = f"https://huggingface.co/{repo}/resolve/{rev}/{fpath}"
        out_path = os.path.join(dest, os.path.basename(fpath))
        if SKIP_EXISTING and file_exists_nonempty(out_path):
            print(f"↪️ Skipping existing: {os.path.basename(out_path)}")
            continue
        if SHOW_PROGRESS:
            hdr = f"-H 'Authorization: Bearer {HF_TOKEN}'" if HF_TOKEN else ""
            ok, _ = run(f"curl -L --fail --retry 5 --retry-delay 2 --retry-all-errors -C - {hdr} --output '{out_path}' '{url}'")
            used_curl = True
            if ok:
                continue
        # Fallback to hf CLI
        ensure_hf_cli()
        rev_flag = f" --revision {revision}" if revision else ""
        ok, _ = run(f"hf download {repo} {fpath}{rev_flag} --local-dir '{dest}'")
        if not ok:
            print(f"❌ HF download failed: {repo}::{fpath}{'@'+revision if revision else ''}{' via curl' if used_curl else ''}")


# CivitAI helpers
SAFE_CHARS = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-_."

def sanitize_filename(name: str) -> str:
    return ''.join(c for c in name if c in SAFE_CHARS) or "civitai_file"


def civitai_resolve_version(mid_or_vid: str, token: str | None) -> tuple[str | None, dict]:
    headers = {"Authorization": f"Bearer {token}"} if token else {}
    # Try as version id
    r = requests.get(f"https://civitai.com/api/v1/model-versions/{mid_or_vid}", headers=headers, timeout=25)
    if r.status_code == 200:
        data = r.json()
        return str(data.get("id")), data
    # Try as model id (get latest version)
    r = requests.get(f"https://civitai.com/api/v1/models/{mid_or_vid}", headers=headers, timeout=25)
    if r.status_code == 200:
        m = r.json()
        versions = m.get("modelVersions") or []
        if versions:
            vid = str(versions[0].get("id"))
            rv = requests.get(f"https://civitai.com/api/v1/model-versions/{vid}", headers=headers, timeout=25)
            return vid, (rv.json() if rv.status_code == 200 else {})
    return None, {}


def civitai_pick_file(version_data: dict) -> tuple[str | None, str]:
    files = version_data.get("files") or []
    primary = next((f for f in files if f.get("primary")), files[0] if files else None)
    if not primary:
        return None, ""
    url = primary.get("downloadUrl")
    name = primary.get("name") or ""
    return url, name


def civitai_dest_for_type(model_type: str) -> str:
    t = (model_type or '').lower()
    d = CONFIG.get('dirs', {})
    if t in {"lora","locon","lycoris"}:
        return d.get('loras')
    if t in {"textualinversion","embedding","textual_inversion"}:
        return d.get('embeddings')
    return d.get('checkpoints')


def civitai_download(item: dict):
    vid_or_mid = str(item.get('id'))
    token = CIVITAI_TOKEN or None
    vid, data = civitai_resolve_version(vid_or_mid, token)
    if not vid:
        print(f"❌ Could not resolve CivitAI id {vid_or_mid}")
        return
    url, name = civitai_pick_file(data)
    if not url:
        print(f"❌ No downloadUrl for version {vid}")
        return
    model = data.get('model') or {}
    mtype = item.get('type') or (model.get('type') if model else None)
    dest = civitai_dest_for_type(mtype)
    if not dest:
        dest = CONFIG['dirs']['checkpoints']
    # Derive short descriptive name
    mname = (model.get('name') or 'civitai')
    prefix = re.sub(r'[^A-Za-z0-9_-]+', '', mname)[:16] or 'civitai'
    ext = ''.join(name.split('.')[-1:]) if '.' in name else 'safetensors'
    filename = sanitize_filename(f"{prefix}_{vid}.{ext}")
    run(f"mkdir -p '{dest}'")
    dl = url
    if 'civitai.com/api/download/models' in dl and token:
        sep = '&' if '?' in dl else '?'
        dl = f"{dl}{sep}token={token}"
    ok, _ = run(f"curl -L --fail --retry 5 --retry-delay 2 --output '{os.path.join(dest, filename)}' '{dl}'")
    if not ok:
        print(f"❌ Download failed for CivitAI {vid_or_mid}")

print("✅ Helpers loaded.")


In [ ]:
# Refined Helpers v2 (robust retries, filenames, revisions)
import math
from typing import Optional, Tuple, Dict, Any

def log(msg: str):
    print(msg)

def run(cmd: str, timeout: int = 1800) -> tuple[bool, str]:
    print(f"$ {cmd}")
    try:
        p = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=timeout)
        out = (p.stdout or '').strip()
        err = (p.stderr or '').strip()
        if p.returncode == 0:
            if out:
                print(out)
            return True, out
        else:
            if err:
                print(err)
            return False, err
    except subprocess.TimeoutExpired:
        return False, f"timeout after {timeout}s: {cmd}"


def fetch_json(url: str, headers: dict | None = None, retries: int = 4, timeout: int = 25) -> tuple[bool, dict]:
    headers = headers or {}
    for i in range(retries):
        try:
            r = requests.get(url, headers=headers, timeout=timeout)
            if r.status_code == 200:
                return True, r.json()
            if r.status_code in (429, 500, 502, 503, 504):
                sleep_s = min(10, 1.5 * (i + 1))
                time.sleep(sleep_s)
                continue
            return False, {}
        except Exception:
            sleep_s = min(10, 1.5 * (i + 1))
            time.sleep(sleep_s)
    return False, {}


def ensure_hf_cli():
    run("pip install -q --upgrade huggingface_hub[cli]")
    hf = shutil.which('hf') or shutil.which('huggingface-cli')
    if not shutil.which('hf') and hf:
        run(f"ln -sf {hf} /usr/local/bin/hf || true")
    if HF_TOKEN:
        os.environ['HUGGINGFACE_HUB_TOKEN'] = HF_TOKEN
        os.environ['HF_TOKEN'] = HF_TOKEN
        run("hf whoami || true")


def hf_download(repo: str, files: list[str], dest: str, revision: Optional[str] = None) -> None:
    ensure_hf_cli()
    run(f"mkdir -p '{dest}'")
    rev_flag = f" --revision {revision}" if revision else ""
    for f in files:
        ok, _ = run(f"hf download {repo} {f}{rev_flag} --local-dir '{dest}'")
        if not ok:
            print(f"❌ HF download failed: {repo}::{f}{'@'+revision if revision else ''}")


SAFE_CHARS = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-_."

def sanitize_filename(name: str) -> str:
    name = ''.join(c for c in name if c in SAFE_CHARS) or "civitai_file"
    if len(name) > 100:
        base, dot, ext = name.rpartition('.')
        base = base[:90] if base else name[:90]
        ext = ext if dot else 'safetensors'
        name = f"{base}.{ext}"
    return name


def civitai_resolve_version(mid_or_vid: str, token: Optional[str]) -> tuple[Optional[str], dict]:
    headers = {"Authorization": f"Bearer {token}"} if token else {}
    ok, data = fetch_json(f"https://civitai.com/api/v1/model-versions/{mid_or_vid}", headers)
    if ok and data.get("id"):
        return str(data["id"]), data
    ok, m = fetch_json(f"https://civitai.com/api/v1/models/{mid_or_vid}", headers)
    if ok:
        versions = m.get("modelVersions") or []
        if versions:
            vid = str(versions[0].get("id"))
            ok2, vd = fetch_json(f"https://civitai.com/api/v1/model-versions/{vid}", headers)
            return vid, (vd if ok2 else {})
    return None, {}


def civitai_pick_file(version_data: dict) -> tuple[Optional[str], str]:
    files = version_data.get("files") or []
    # Prefer safetensors, then others
    preferred = None
    for fobj in files:
        n = (fobj.get("name") or "").lower()
        if n.endswith(".safetensors"):
            preferred = fobj
            break
    if not preferred and files:
        preferred = files[0]
    if not preferred:
        return None, ""
    return preferred.get("downloadUrl"), preferred.get("name") or ""


def civitai_dest_for_type(model_type: Optional[str]) -> str:
    t = (model_type or '').lower()
    d = CONFIG.get('dirs', {})
    if t in {"lora","locon","lycoris"}: return d.get('loras')
    if t in {"textualinversion","embedding","textual_inversion"}: return d.get('embeddings')
    return d.get('checkpoints')


def civitai_download(item: dict):
    vid_or_mid = str(item.get('id'))
    token = CIVITAI_TOKEN or None
    vid, data = civitai_resolve_version(vid_or_mid, token)
    if not vid:
        print(f"❌ Could not resolve CivitAI id {vid_or_mid}")
        return
    url, name = civitai_pick_file(data)
    if not url:
        print(f"❌ No downloadUrl for version {vid}")
        return
    model = data.get('model') or {}
    mtype = item.get('type') or (model.get('type') if model else None)

    # Destination resolution with override support
    dest = item.get('dest')
    if dest:
        # dest can be a key in dirs or an absolute path
        if not dest.startswith('/'):
            dest = CONFIG['dirs'].get(dest, CONFIG['dirs']['checkpoints'])
    else:
        dest = civitai_dest_for_type(mtype)
        if not dest:
            dest = CONFIG['dirs']['checkpoints']

    # Filename override or derived
    override_name = item.get('filename')
    if override_name:
        filename = sanitize_filename(override_name)
    else:
        mname = (model.get('name') or 'civitai')
        prefix = re.sub(r'[^A-Za-z0-9_-]+', '', mname)[:16] or 'civitai'
        ext = (name.rsplit('.', 1)[-1] if '.' in name else 'safetensors')
        filename = sanitize_filename(f"{prefix}_{vid}.{ext}")

    run(f"mkdir -p '{dest}'")

    # Token append on initial CivitAI URL only
    dl = url
    if 'civitai.com/api/download/models' in dl and token:
        sep = '&' if '?' in dl else '?'
        dl = f"{dl}{sep}token={token}"

    # Resume and robust curl
    ok, _ = run(
        f"curl -L --fail --retry 5 --retry-delay 2 --retry-all-errors -C - --output '{os.path.join(dest, filename)}' '{dl}'"
    )
    if not ok:
        print(f"❌ Download failed for CivitAI {vid_or_mid}")

def normalize_model_dirs():
    # Move any mistakenly created /workspace/<name> dirs into CONFIG['dirs'][name]
    names = ["checkpoints","unet","clip","clip_vision","style_models","vae","loras","embeddings","sam2"]
    for name in names:
        src = f"/workspace/{name}"
        dst = CONFIG.get('dirs', {}).get(name)
        try:
            if not dst:
                continue
            if os.path.abspath(src) == os.path.abspath(dst):
                continue
            if os.path.isdir(src):
                os.makedirs(dst, exist_ok=True)
                for entry in os.listdir(src):
                    src_path = os.path.join(src, entry)
                    dst_path = os.path.join(dst, entry)
                    try:
                        if os.path.isdir(src_path) and not os.path.exists(dst_path):
                            shutil.move(src_path, dst_path)
                        elif os.path.isfile(src_path) and not os.path.exists(dst_path):
                            shutil.move(src_path, dst_path)
                    except Exception as move_err:
                        print(f"⚠️ Move warning for {src_path} -> {dst_path}: {move_err}")
                # Try remove empty src
                try:
                    os.rmdir(src)
                except Exception:
                    pass
        except Exception as e:
            print(f"⚠️ Normalize warning for {name}: {e}")

print("✅ Refined helpers loaded.")


In [ ]:
# Config hotfix: normalize HF destinations and remove invalid FLUX.1-dev entries
fixed = []
for it in CONFIG.get("huggingface_models", []):
    repo = it.get("repo", "")
    files = list(it.get("files") or [])
    dest = it.get("dest")
    if repo == "black-forest-labs/FLUX.1-dev":
        # Keep only the VAE file which is valid; drop missing UNET/encoders from this repo
        files = [f for f in files if f == "ae.safetensors"]
        if not files:
            continue
        it = {**it, "files": files, "dest": "vae"}
    fixed.append(it)
CONFIG["huggingface_models"] = fixed

print("HF models after cleanup:")
for it in CONFIG["huggingface_models"]:
    print(" -", it)

print("✅ Hotfix applied. Now run 'Apply Configuration v2'.")


In [ ]:
# Apply Configuration v2 (validation + revisions)
print("Applying configuration (v2)...")
ensure_dirs()

# Helper to resolve dest keys to absolute paths
def resolve_dest(dest_key_or_path: str | None) -> str:
    if not dest_key_or_path:
        return CONFIG['dirs']['checkpoints']
    if isinstance(dest_key_or_path, str) and dest_key_or_path.startswith("/"):
        return dest_key_or_path
    return CONFIG['dirs'].get(dest_key_or_path, CONFIG['dirs']['checkpoints'])

# Validate git presence for custom nodes
have_git = shutil.which('git') is not None
if not have_git and CONFIG.get('custom_nodes'):
    print("⚠️ git not found; custom nodes cloning will be skipped.")

# HuggingFace models (support per-item revision)
for item in CONFIG.get("huggingface_models", []):
    repo = item.get("repo")
    files = item.get("files") or []
    dest = resolve_dest(item.get("dest"))
    revision = item.get("revision")
    if not repo or not files:
        print(f"⚠️ Skipping invalid HF entry: {item}")
        continue
    print(f"➡️ HF: {repo}{'@'+revision if revision else ''} -> {dest}")
    hf_download(repo, files, dest, revision=revision)

# CivitAI items (process smaller items first)
items = CONFIG.get("civitai_items", [])
small_first = [it for it in items if str(it.get('type','')).lower() != 'checkpoint']
checkpoints = [it for it in items if str(it.get('type','')).lower() == 'checkpoint']

if small_first:
    print(f"➡️ CivitAI (LoRAs/Embeddings first): {len(small_first)} item(s)")
    for item in small_first:
        try:
            if 'id' not in item:
                print(f"⚠️ Skipping invalid CivitAI entry (missing id): {item}")
                continue
            print(f"   • {item}")
            civitai_download(item)
        except Exception as e:
            print(f"❌ CivitAI error: {e}")

if checkpoints:
    print(f"➡️ CivitAI (Checkpoints next): {len(checkpoints)} item(s)")
    for item in checkpoints:
        try:
            if 'id' not in item:
                print(f"⚠️ Skipping invalid CivitAI entry (missing id): {item}")
                continue
            print(f"   • {item}")
            civitai_download(item)
        except Exception as e:
            print(f"❌ CivitAI error: {e}")

# Custom nodes
if CONFIG.get("custom_nodes") and have_git:
    base = resolve_dest('custom_nodes')
    run(f"mkdir -p '{base}'")
    for node in CONFIG['custom_nodes']:
        repo = node.get('repo')
        if not repo:
            print(f"⚠️ Skipping custom node without repo: {node}")
            continue
        name = repo.rstrip('/').split('/')[-1].replace('.git','')
        path = os.path.join(base, name)
        if os.path.isdir(path):
            print(f"🔄 Updating {name}")
            run(f"cd '{path}' && git pull --ff-only | cat")
        else:
            print(f"⬇️ Cloning {name}")
            run(f"git clone '{repo}' '{path}' | cat")

# Final normalization pass (if any tools wrote to /workspace/<name>)
normalize_model_dirs()
print("\n✅ Done (v2).")


In [ ]:
# Verify Downloads
import glob

base = CONFIG.get('dirs', {})

def list_dir(label, path, patterns=("*.safetensors","*.pt","*.ckpt","*.bin","*.gguf")):
    if not path:
        return
    files = []
    for pat in patterns:
        files.extend(glob.glob(os.path.join(path, pat)))
    if files:
        total = sum(os.path.getsize(f) for f in files) / (1024*1024)
        print(f"✅ {label}: {len(files)} files, {total:.1f} MB")
        for f in sorted(files)[:10]:
            sz = os.path.getsize(f)/(1024*1024)
            print(f" - {os.path.basename(f)} ({sz:.1f} MB)")
        if len(files) > 10:
            print(f" ... +{len(files)-10} more")
    else:
        print(f"❌ {label}: none")

list_dir("Checkpoints", base.get('checkpoints'))
list_dir("UNet", base.get('unet'))
list_dir("CLIP", base.get('clip'))
list_dir("CLIP Vision", base.get('clip_vision'))
list_dir("Style Models", base.get('style_models'))
list_dir("VAE", base.get('vae'))
list_dir("LoRAs", base.get('loras'))
list_dir("Embeddings", base.get('embeddings'))
list_dir("SAM2", base.get('sam2'))

print("\n🔍 Verification complete.")
